In [1]:
import pandas
import numpy as np
import modin.pandas as pd
import dask.dataframe as dd

In [2]:
# import ray
# ray.init()

In [3]:
# import os
# os.environ["MODIN_ENGINE"] = "ray"

In [4]:
#modin and pandas were unable to load the dataset.My machine was hanging

In [5]:
%%time
df = dd.read_csv('data.csv')
df

CPU times: user 14.1 ms, sys: 587 µs, total: 14.7 ms
Wall time: 233 ms


,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
npartitions=89,,,,,,,,,
,object,object,int64,int64,object,object,float64,int64,object
,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...


In [6]:
df.shape

(Delayed('int-fdd97adc-c25d-4512-9a21-71fc0fa20313'), 9)

In [7]:
%%writefile file.yaml
file_type : 'csv'
file_name : 'data'
inbound_delimiter : ","
outbound_delimiter : "|"
skip_leading_rows : 1
columns : ['event_time','event_type','category_id','category_code','product_id','brand','price','user_id','user_session']
    

Overwriting file.yaml


In [8]:
%%writefile test.py
import yaml
import logging


def config_file(file):
    with open(file,'r') as myfile:
        try:
            return yaml.safe_load(myfile)
        except yaml.YAMLError as exc:
            logging.error(exc)
            
            
def col_validation(df,table_config):
    
    config_cols = list(map(lambda x: x.lower(),table_config['columns']))
    config_cols.sort()
    df.columns = list(map(lambda x: x.lower(),list(df.columns)))    
    if len(df.columns)== len(config_cols) and list(config_cols).sort() == list(df.columns).sort():
        print('Column name and column length validation passed')
        return 1
    else:
        print('Column name and column length validation failed')
        mismatched_columns_file = list(set(df.columns).difference(config_cols))
        print("Following columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(config_cols).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df.columns: {df.columns}')
        logging.info(f'config_cols: {config_cols}')
        return 0

Overwriting test.py


In [9]:
import test
config_data = test.config_file('file.yaml')

In [10]:
config_data

{'file_type': 'csv',
 'file_name': 'data',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['event_time',
  'event_type',
  'category_id',
  'category_code',
  'product_id',
  'brand',
  'price',
  'user_id',
  'user_session']}

In [11]:
file = config_data['file_type']
file

'csv'

In [12]:
src_file = "./" + config_data['file_name'] + f'.{file}'

In [13]:
df = dd.read_csv(src_file)
#df = df.compute() #converting a dask dataframe to a pandas dataframe
df

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
npartitions=89,,,,,,,,,
,object,object,int64,int64,object,object,float64,int64,object
,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...


In [14]:
test.col_validation(df,config_data)

Column name and column length validation passed


1

In [15]:
if test.col_validation(df,config_data) == 1:
    import csv
    
    with open(src_file,'r') as csv_file:
        reader = csv.reader(csv_file)
        
        with open('new_data.txt','w') as newfile:
            writer = csv.writer(newfile,delimiter = "|")
            
            for line in reader:
                writer.writerow(line)

Column name and column length validation passed


In [17]:
import gzip
import shutil
with open('new_data.txt', 'rb') as f_in:
    with gzip.open('myfile.gz', 'w',compresslevel = 9) as f_out:
        shutil.copyfileobj(f_in, f_out)
